# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import os
os.chdir('..')

from nn.models import Models
from bots import BotBid
from bidding import bidding
from util import hand_to_str
from deck52 import random_deal
from sample import Sample
import conf

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('./config/default.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('./config/default.conf'))  # Load sampling strategies

INFO:tensorflow:Restoring parameters from ..\models/gib21_model/gib21-1000000
INFO:tensorflow:Restoring parameters from ..\models/gib21_info_model/gib21_info-500000
INFO:tensorflow:Restoring parameters from ..\models/lead_model_b/lead-1000000
INFO:tensorflow:Restoring parameters from ..\models/lr3_model/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\models/lefty_model/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\models/dummy_model/dummy-920000
INFO:tensorflow:Restoring parameters from ..\models/righty_model/righty-1000000
INFO:tensorflow:Restoring parameters from ..\models/decl_model/decl-1000000


### Running through an example

In [3]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = False, False

# you sit West and hold:
hand = 'AK.JT4.AQT.KQJ76'

# the auction goes:
# (a few words about 'PAD_START':
# the auction is padded to dealer North
# if North is not dealer, than we have to put in a 'PAD_START' for every seat that was skipped
# if East deals we have one pad (because North is skipped)
# if South deals we have two pads (because North and East are skipped)
# etc.)
auction = ['PAD_START', 'PAD_START', 'PAD_START']

# what's your bid?

In [4]:
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, -1, -1, 0.1, sampler, False)

bot_bid.bid(auction).bid


'2N'

In [5]:
# you double, North bids 2S, your partner 3H, PASS, back to you ...

bid = bot_bid.bid(['PAD_START', 'PAD_START', 'PAD_START','2N','PASS','3C','PASS'])
bid.bid

'3D'

In [6]:
# what options were considered?

bid.to_dict()['candidates']

[{'call': '3D', 'insta_score': 0.9975}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [7]:
bid.samples

['J9xx.9xxx.Kx.A9x QT8x.AQ8x.Jxx.8x xxx.Kx.98xxx.Txx AK.JTx.AQT.KQJxx',
 'Txx.Axxx.K9xx.Tx QJ9x.Q98x.Jx.A98 8xxx.Kx.8xxx.xxx AK.JTx.AQT.KQJxx',
 'QJx.Axxx.KJx.9xx 98xx.KQ9x.xxx.AT Txxx.8x.98xx.8xx AK.JTx.AQT.KQJxx',
 'JT9xx.Qxx.K9.98x Q8.AK98.Jxxx.xxx xxxx.xxx.8xxx.AT AK.JTx.AQT.KQJxx',
 'Txxx.A9xx.Kx.T8x QJ8x.KQxx.8xx.xx 9xx.8x.J9xxx.A9x AK.JTx.AQT.KQJxx',
 'QJ9x.AQxx.98x.xx T8xx.K98x.KJx.8x xxx.xx.xxxx.AT9x AK.JTx.AQT.KQJxx',
 'Q9xx.A8xx.KJx.9x T8xx.KQxx.9xx.Ax Jxx.9x.8xxx.T8xx AK.JTx.AQT.KQJxx',
 'Q9xx.A9xx.Jxx.8x xxx.KQ8x.K8x.9xx JT8x.xx.9xxx.ATx AK.JTx.AQT.KQJxx',
 'Qxx.KQ9xx.9x.Axx J98x.A8x.Kxxx.Tx Txxx.xx.J8xx.98x AK.JTx.AQT.KQJxx',
 'Q8xx.A8xx.Kx.T98 Jxx.Q9xx.xxx.Axx T9xx.Kx.J98xx.xx AK.JTx.AQT.KQJxx',
 'J8xx.A98x.Jxx.Ax Q9.KQxx.K98.98xx Txxxx.xx.xxxx.Tx AK.JTx.AQT.KQJxx',
 'J8x.Q8xx.Kxx.T8x Qxxx.A9xx.J9x.xx T9xx.Kx.8xxx.A9x AK.JTx.AQT.KQJxx',
 '98x.AKQx.Jxx.98x QJxx.98xx.K8x.xx Txxx.xx.9xxx.ATx AK.JTx.AQT.KQJxx',
 'Txx.AKQx.9xxx.xx J98x.8xxx.K8.Axx Qxxx.9x.Jxxx.T98 AK.JTx.AQT.

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [8]:
# we got some samples above. if we want more, we can always get more

sample_hands = bot_bid.sample_hands(['PAD_START', 'PAD_START', 'PAD_START','2N','PASS','3C','PASS','3D'])
print(sample_hands.shape[0])
for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

0


How do we know on which sample 4H is making and on which sample it is not? It's the topic for another tutorial, but basically we could do either (1) solve the sample double dummpy or (2) estimate the outcome using a neural network

### Bidding through an entire auction for all four hands

Now we'll take a random deal and let the bot bid for each hand. We assume North deals and nobody is vulnerable.

In [9]:
hands = random_deal().split()
hands # N, E, S, W

['9.AT7.QJ95.97432',
 'K8532.Q9.T3.AKT6',
 'J76.K54.A84.QJ85',
 'AQT4.J8632.K762.']

In [10]:
bidder_bots = [BotBid([False, False], hand, models, -1, -1, 0.1, sampler, False) for hand in hands]

In [11]:
auction = []  # since North deals, we don't need any 'PAD_START'

turn_i = 0  # whose turn is it to bid

while not bidding.auction_over(auction):
    auction.append(bidder_bots[turn_i].bid(auction).bid)
    turn_i = (turn_i + 1) % 4  # next player's turn
    
auction

['4S', 'PASS', 'PASS']


['PASS', '1S', 'PASS', '4C', 'PASS', '4S', 'PASS', 'PASS', 'PASS']